### 📐 LLL Algorithm: Wikipedia Implementation & Tests

This implementation of the Lenstra–Lenstra–Lovász (LLL) lattice basis reduction algorithm is based on the description from [Wikipedia](https://en.wikipedia.org/wiki/Lenstra%E2%80%93Lenstra%E2%80%93Lov%C3%A1sz_lattice_basis_reduction_algorithm).

The algorithm has been implemented in Python and extended with custom test cases to verify correctness, lattice preservation, and reduced vector lengths across various dimensions.

In [11]:
import numpy as np


def gram_schmidt(basis):
    n = len(basis)
    dim = len(basis[0])

    ortho = [np.zeros(dim) for _ in range(n)]
    mu = np.zeros((n, n))

    for i in range(n):
        b_i = basis[i]
        projection = np.zeros(dim,  dtype=object)

        for j in range(i):
            mu[i, j] = np.dot(b_i, ortho[j]) / np.dot(ortho[j], ortho[j])
            projection += mu[i, j] * ortho[j]

        ortho[i] = b_i - projection

    return ortho, mu



def lll_reduce(basis, delta=0.75, verbose=False):

    basis = [b.copy() for b in basis]
    n = len(basis)
    k = 1


    while k < n:
        ortho, mu = gram_schmidt(basis)
        for j in range(k - 1, -1, -1):  # j < k
            if abs(mu[k, j]) > 0.5:
                r = round(mu[k, j])
                basis[k] -= r * basis[j]
                ortho, mu = gram_schmidt(basis)

        norm_sq_prev = np.dot(ortho[k - 1], ortho[k - 1])
        norm_sq_curr = np.dot(ortho[k], ortho[k])

        lhs = delta * norm_sq_prev
        rhs = norm_sq_curr + mu[k, k - 1]**2 * norm_sq_prev

        if lhs > rhs:
            basis[k], basis[k - 1] = basis[k - 1], basis[k].copy()

            k = max(k - 1, 1)
        else:
            k += 1

    return basis





### ✅ LLL Reduction Tests (2D)

We test whether different 2D basis reduction methods (e.g. LLL, classical) produce equivalent bases.

Bases are normalized (up to sign and order) before comparison.

Matches confirm correct lattice reduction.

In [16]:
import import_ipynb
import basis_reduction_2d as br2d; # type: ignore ## static funcs error?

def bases_equal_2d(basis1, basis2):
    def normalize(v):
        v = np.array(v)
        return v if v[0] > 0 or (v[0] == 0 and v[1] >= 0) else -v

    def sorted_basis(u, v):
        return sorted([normalize(u).tolist(), normalize(v).tolist()])

    return sorted_basis(basis1[0], basis1[1]) == sorted_basis(basis2[0], basis2[1])

def tests_brlll(basis_list, verbose=False):

    tests_amount = len(basis_list)
    tests_passed = 0

    results = []

    for i, basis in enumerate(basis_list):
        reduced = lll_reduce(basis)

        same = br2d.check_same_lattice(*basis, *reduced)

        original_len = min(np.linalg.norm(v) for v in basis)
        reduced_len = min(np.linalg.norm(v) for v in reduced)
        improved = reduced_len <= original_len

        result = int(same and improved)
        if result:
            tests_passed += 1

        if verbose:
            print(f"{'✅' if result else '❌'} Test {i + 1}: {'PASSED' if result else 'FAILED'}")
            print("Initial basis:")
            for v in basis:
                print(" ", v)
            print("Reduced basis:")
            for v in reduced:
                print(" ", v)
            print()

        results.append({
            "basis": [v.tolist() for v in reduced],
            "result": result
        })

    if verbose:
        print(f"\n📊 {tests_passed}/{tests_amount} tests passed.")

    return results


basis_list = br2d.generate_n_2d_bases(10)
test_results_br2d = br2d.tests_br2d(basis_list)
test_results_brlll = tests_brlll(basis_list)
tests_amount = 10
tests_passed = 0

for i in range(len(test_results_br2d)):
    res_2d = test_results_br2d[i]
    res_lll = test_results_brlll[i]

    b1_2d, b2_2d = res_2d["b1"], res_2d["b2"]
    b1_lll, b2_lll = res_lll["basis"]
    match = bases_equal_2d([b1_2d,b2_2d], [b1_lll, b2_lll])

    if match:
        print(f"Test {i+1}: ✅ MATCH")
        print(f"  🔹 br2d  → b1 = {b1_2d}, b2 = {b2_2d}")
        print(f"  🔸 brlll → b1 = {b1_lll}, b2 = {b2_lll}")
        tests_passed+=1
    else:
        print(f"Test {i+1}: ❌ DIFFERENT")
        print(f"  🔹 br2d  → b1 = {b1_2d}, b2 = {b2_2d}")
        print(f"  🔸 brlll → b1 = {b1_lll}, b2 = {b2_lll}")

print(f"\n📊 {tests_passed}/{tests_amount} tests passed.")




Test 1: ✅ MATCH
  🔹 br2d  → b1 = [-18  13], b2 = [15 29]
  🔸 brlll → b1 = [-18, 13], b2 = [15, 29]
Test 2: ✅ MATCH
  🔹 br2d  → b1 = [-21  20], b2 = [32 34]
  🔸 brlll → b1 = [-21, 20], b2 = [32, 34]
Test 3: ✅ MATCH
  🔹 br2d  → b1 = [35  5], b2 = [ 17 -44]
  🔸 brlll → b1 = [35, 5], b2 = [17, -44]
Test 4: ✅ MATCH
  🔹 br2d  → b1 = [ 5 10], b2 = [16 -7]
  🔸 brlll → b1 = [5, 10], b2 = [16, -7]
Test 5: ✅ MATCH
  🔹 br2d  → b1 = [-19 -14], b2 = [-26  46]
  🔸 brlll → b1 = [-19, -14], b2 = [-26, 46]
Test 6: ✅ MATCH
  🔹 br2d  → b1 = [ 24 -22], b2 = [-39 -26]
  🔸 brlll → b1 = [24, -22], b2 = [-39, -26]
Test 7: ✅ MATCH
  🔹 br2d  → b1 = [ 4 -7], b2 = [9 6]
  🔸 brlll → b1 = [-4, 7], b2 = [9, 6]
Test 8: ✅ MATCH
  🔹 br2d  → b1 = [14  9], b2 = [-35  48]
  🔸 brlll → b1 = [14, 9], b2 = [35, -48]
Test 9: ✅ MATCH
  🔹 br2d  → b1 = [-37  31], b2 = [39 48]
  🔸 brlll → b1 = [-37, 31], b2 = [39, 48]
Test 10: ✅ MATCH
  🔹 br2d  → b1 = [-9 -7], b2 = [-17  31]
  🔸 brlll → b1 = [-9, -7], b2 = [-17, 31]

📊 10/10 tests 

### 🔓 LLL Attack

Assume the message has the form m = B + x, where x is a small unknown.

Steps:
- Construct the polynomial: \\( f(T) = (B + T)^e - c \; mod n \\)
- Build a lattice using scaled basis vectors
- Apply LLL to find a short vector interpreted as \\( g(T) \\)
- Solve \\( g(x) = 0 \\) over the integers
- Recover \\( m = B + x \\) and verify \\( pow(m, e, n) == c \\)



In [17]:
from sympy import symbols, Poly, solve, im

n = 1927841055428697487157594258917
e = 3
Y = 100 #   |x| <= Y (xmax)
B = 200805000114192305180009190000
c = 30326308498619648559464058932

print("🔐 RSA Setup:")
print(f"   • Public modulus (n): {n}")
print(f"   • Public exponent (e): {e}")
print(f"   • Ciphertext (c): {c}")
print()
print("📦 Message structure:")
print(f"   • Assumed form: m = B + x")
print(f"   • Known prefix (B): {B}")
print(f"   • Unknown root (x): ?")
print(f"   • Bound on x: |x| <= {Y}")


a0 = pow(B,3,n) - c
a1 = (3 * pow(B,2,n)) % n
a2 = (3 * B) % n
a3 = 1

#print(a0, a1, a2, a3)


v1 = np.array([n, 0, 0, 0])
v2 = np.array([0, n * Y, 0, 0])
v3 = np.array([0, 0, n * pow(Y,2,n), 0])
v4 = np.array([a0, a1*Y, a2*pow(Y,2,n), a3*pow(Y,3,n)])


basis = [v1, v2, v3, v4]

reduced = lll_reduce(basis)  # <- твоя функция
coeffs = reduced[0]


# print(v1)
# print(v2)
# print(v3)
# print(v4)

e0 = int(coeffs[0])
e1 = int(coeffs[1]) // Y
e2 = int(coeffs[2]) // (Y**2)
e3 = int(coeffs[3]) // (Y**3)

T = symbols('T')
g = Poly(e3*T**3 + e2*T**2 + e1*T + e0)
roots = solve(g)

c_check = 0
root = 0

for r in roots:
    if(im(r) != 0 or r >= B or r < 0):
        continue
    else:
        root = r
        c_check = pow((B+r),e,n)

print()
print("🧾 Final Result Summary")
print("────────────────────────────────────────────")

print(f"📍 Recovered root x: {root}")
print(f"📍 Reconstructed message m = B + x = {B} + {root} = {B + root}\n")
print(f"🧮 Computed ciphertext: c' = pow(m, e, n) = {c_check}")
print(f"🎯 Target ciphertext:    c  = {c}")
print()

if c_check == c:
    print("✅ SUCCESS: LLL attack successfully recovered the correct message!")
else:
    print("❌ FAILURE: Recovered message does not match the original ciphertext.")


🔐 RSA Setup:
   • Public modulus (n): 1927841055428697487157594258917
   • Public exponent (e): 3
   • Ciphertext (c): 30326308498619648559464058932

📦 Message structure:
   • Assumed form: m = B + x
   • Known prefix (B): 200805000114192305180009190000
   • Unknown root (x): ?
   • Bound on x: |x| <= 100

🧾 Final Result Summary
────────────────────────────────────────────
📍 Recovered root x: 42
📍 Reconstructed message m = B + x = 200805000114192305180009190000 + 42 = 200805000114192305180009190042

🧮 Computed ciphertext: c' = pow(m, e, n) = 30326308498619648559464058932
🎯 Target ciphertext:    c  = 30326308498619648559464058932

✅ SUCCESS: LLL attack successfully recovered the correct message!


In [10]:
# basis = [
#     np.array([1, -1, 3]),
#     np.array([1,  0, 5]),
#     np.array([1,  2, 6])
# ]
#
# test_results_br2d = lll_reduce(basis)
# print(test_results_br2d)